Positional Encoding

In [5]:
import numpy as np
import math
def positional_encoding(max_len, d_model):
    # max_len: độ dài tối đa của 1 chuỗi
    # d_model: kích thước embedding
    pe = np.zeros((max_len, d_model))
    for pos in range(max_len):
        for i in range(0, d_model, 2):
            angle = pos / (10000 ** (i / d_model))
            # Vị trí chẵn
            pe[pos, i] = math.sin(angle)
            # Vị trí lẻ
            pe[pos, i+1] = math.cos(angle)
    return pe



In [6]:
max_len = 5
d_model = 4
pe = np.zeros((5, 4))
print(f"ma trận ban đầu:\n{pe}")
print(f"------------------")
for pos in range(max_len):
    for i in range(0, d_model, 2):
        angle = pos / (10000 ** (i / d_model))
        pe[pos][i] = math.sin(angle)
        pe[pos][i+1] = math.cos(angle)
print(f"ma trận positional encoding:\n{pe}")
print(f"--------------------------")
for i in range(max_len):
    print(f"vector PE cho vị trí thứ: {i} là:\n{pe[i]}")
    print(f"------------------------------")


ma trận ban đầu:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
------------------
ma trận positional encoding:
[[ 0.          1.          0.          1.        ]
 [ 0.84147098  0.54030231  0.00999983  0.99995   ]
 [ 0.90929743 -0.41614684  0.01999867  0.99980001]
 [ 0.14112001 -0.9899925   0.0299955   0.99955003]
 [-0.7568025  -0.65364362  0.03998933  0.99920011]]
--------------------------
vector PE cho vị trí thứ: 0 là:
[0. 1. 0. 1.]
------------------------------
vector PE cho vị trí thứ: 1 là:
[0.84147098 0.54030231 0.00999983 0.99995   ]
------------------------------
vector PE cho vị trí thứ: 2 là:
[ 0.90929743 -0.41614684  0.01999867  0.99980001]
------------------------------
vector PE cho vị trí thứ: 3 là:
[ 0.14112001 -0.9899925   0.0299955   0.99955003]
------------------------------
vector PE cho vị trí thứ: 4 là:
[-0.7568025  -0.65364362  0.03998933  0.99920011]
------------------------------


Sau khi tính PE, ta cộng vào embedding ban đầu: X' = X + PE

In [7]:
# Giả sử vector X ban đầu có giá trị:
x = np.random.randn(max_len, d_model)
print(f"vector embedding của input:\n{x}")
x = x + pe
print(f"---------------------------------")
print(f"vector x khi thêm positional encoding:\n{x}")

vector embedding của input:
[[-0.50717403  1.99720352  1.50465625  0.36161226]
 [ 1.16133027 -0.2258359  -0.47576192 -1.97274738]
 [-0.01531425 -0.45071204 -0.5185215   0.31283653]
 [-0.12794477  0.27173949  0.38107136 -0.57542029]
 [-0.64704955  1.33265802  0.90629939  0.31305582]]
---------------------------------
vector x khi thêm positional encoding:
[[-0.50717403  2.99720352  1.50465625  1.36161226]
 [ 2.00280125  0.3144664  -0.46576209 -0.97279738]
 [ 0.89398317 -0.86685888 -0.49852284  1.31263653]
 [ 0.01317524 -0.71825301  0.41106686  0.42412974]
 [-1.40385205  0.6790144   0.94628873  1.31225593]]


Multihead Attention

Residual and Normalization Layer

In [15]:
import torch
import torch.nn as nn
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        # gamma và beta
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zero(d_model))
        self.eps = eps
    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased= False)

        x_hat = (x - mean) / torch.sqrt(var + self.eps)
        return self.gamma * x_hat + self.beta

In [29]:
d_model = 5
gamma1 = nn.Parameter(torch.zeros(d_model))
gamma2 = torch.zeros(d_model)
print(f"gamma1:\n{gamma1}")
print(f"gamma2:\n{gamma2}")
print(f"---------------------------")
matrix = torch.zeros(3,4)
print(matrix)

gamma1:
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
gamma2:
tensor([0., 0., 0., 0., 0.])
---------------------------
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [34]:
# Tạo input gồm 2 câu batch_size = 2
batch_size = 2
seq_len = 3
d_model = 5
x = torch.randn(batch_size,seq_len, d_model)
print(x)
mean = x.mean(dim=-1, keepdim=True)
print(f"Trung bình của từng hàng: \n{mean}")
var = x.var(dim = -1, keepdim=True, unbiased=False)
print(f"Phương sai của từng hàng:\n{var}")

tensor([[[-0.7275,  0.6259, -0.7753, -0.4079,  0.8510],
         [ 2.2518, -0.4178,  1.7594, -0.0621, -2.1916],
         [-0.8125, -0.8072,  1.3223,  0.1695, -0.9045]],

        [[ 0.1356,  1.7001, -0.8230, -1.1005,  0.6589],
         [ 0.5328,  1.6004,  0.6010,  0.7147,  0.2658],
         [ 0.1755,  0.8646, -0.5448,  0.5204,  1.2276]]])
Trung bình của từng hàng: 
tensor([[[-0.0868],
         [ 0.2679],
         [-0.2065]],

        [[ 0.1142],
         [ 0.7429],
         [ 0.4487]]])
Phương sai của từng hàng:
tensor([[[0.4750],
         [2.5578],
         [0.7388]],

        [[1.0332],
         [0.2056],
         [0.3693]]])


In [38]:
gamma = torch.randn(d_model)
print(f"gamma:\n{gamma}")
x_hat = (x - mean) / torch.sqrt(var + 1e-6)
print(f"Chuẩn hóa các giá trị của input:\n{x_hat}")
ans = gamma * x_hat + gamma
print(f"Đầu ra layernorm của input:\n{ans}")

gamma:
tensor([-0.6576, -0.8526, -1.4558, -0.1420,  0.0609])
Chuẩn hóa các giá trị của input:
tensor([[[-0.9297,  1.0340, -0.9990, -0.4659,  1.3607],
         [ 1.2404, -0.4288,  0.9326, -0.2064, -1.5379],
         [-0.7050, -0.6989,  1.7786,  0.4374, -0.8121]],

        [[ 0.0211,  1.5602, -0.9220, -1.1951,  0.5358],
         [-0.4634,  1.8910, -0.3131, -0.0623, -1.0523],
         [-0.4495,  0.6844, -1.6348,  0.1181,  1.2818]]])
Đầu ra layernorm của input:
tensor([[[-4.6218e-02, -1.7342e+00, -1.4147e-03, -7.5856e-02,  1.4367e-01],
         [-1.4734e+00, -4.8705e-01, -2.8134e+00, -1.1272e-01, -3.2737e-02],
         [-1.9398e-01, -2.5671e-01, -4.0451e+00, -2.0415e-01,  1.1434e-02]],

        [[-6.7149e-01, -2.1829e+00, -1.1352e-01,  2.7705e-02,  9.3474e-02],
         [-3.5290e-01, -2.4650e+00, -9.9995e-01, -1.3318e-01, -3.1818e-03],
         [-3.6201e-01, -1.4362e+00,  9.2410e-01, -1.5879e-01,  1.3887e-01]]])
